In [ ]:
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("PiC/phrase_similarity")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/7004 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Features that could be important:

- Greedy word-wise similarity of phrase
- Lesk overlap with content words in context sentence


In [ ]:
import pandas as pd


train = pd.DataFrame(dataset['train'])
test = pd.DataFrame(dataset['test'])
val = pd.DataFrame(dataset['validation'])

In [ ]:
train

,phrase1,phrase2,sentence1,sentence2,label,idx
0,newly formed camp,recently made encampment,newly formed camp is released from the membran...,recently made encampment is released from the ...,0,0
1,one data,a particular statistic,"According to one data, in 1910, on others – in...","According to a particular statistic, in 1910, ...",1,1
2,particular structure,specific edifice,Note that Fact 1 does not assume any particula...,Note that Fact 1 does not assume any specific ...,0,2
3,involved people,participating individuals,Assessment-Center are usually group-processes ...,Assessment-Center are usually group-processes ...,1,3
4,different cross,opposing inquiries,"At the end of the 1980s, a different cross had...","At the end of the 1980s, a opposing inquiries ...",0,4
...,...,...,...,...,...,...
6999,similar notice,comparable notification,Nereus and Achilleus a similar notice was admi...,Nereus and Achilleus a comparable notification...,1,6999
7000,color map,painted chart,Color quantization is the process of creating ...,Color quantization is the process of creating ...,0,7000
7001,dutch tool,device from the people of the Netherlands,petra is a dutch tool with which patients and ...,petra is a device from the people of the Nethe...,1,7001
7002,secondary concern,less serious issue,"However, granting this access, especially to a...","However, granting this access, especially to a...",1,7002


In [ ]:
import spacy
!python -m spacy download en_core_web_lg

nlp = spacy.load("en_core_web_lg")
doc = nlp("Assessment-Center are usually group-processes with high validity and acceptance of the involved people.")
for token in doc:
    print(token.text, token.pos_, token.dep_)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Assessment NOUN compound
- PUNCT punct
Center PROPN nsubj
are AUX ROOT
usually ADV advmod
group NOUN compound
- PUNCT punct
processes NOUN attr
with ADP prep
high ADJ amod
validity NOUN pobj
and CCONJ cc
acceptance NOUN conj
of ADP prep
the DET det
involved ADJ amod
people NOUN pobj
. PUNCT punct


In [ ]:
import nltk
nltk.download('brown')

from nltk.corpus import brown    # Consider changing corpus (more tokens, etc.)

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


In [ ]:
import gensim as gs
from gensim.models import FastText

md = FastText(brown.sents(), min_count=1, vector_size=100, window=5)

In [ ]:
p1 = 'former charge'
p2 = 'previous responsibility'

def avg_sim(p1, p2):
  p1_nlp = nlp(p1)
  p2_nlp = nlp(p2)


  avg_similarity = 0.0
  for t1 in p1_nlp:
    max_similarity = 0.0
    if t1.tag_ in ['PUNCT']:
      continue
    for t2 in p2_nlp:
      if t2.tag_ in ['PUNCT']:
        continue
      sim = md.wv.similarity(t1.text, t2.text)
      if sim > max_similarity:
        max_similarity = sim
    avg_similarity += max_similarity
    # print(avg_similarity)

  avg_similarity /= len([token for token in p1_nlp if token.tag_ != 'PUNCT'])
  if avg_similarity > 1:
    return 1

  return avg_similarity

In [ ]:
avg_sim(p1, p2)

0.8469060063362122
1.7855841517448425


0.8927920758724213

In [ ]:
avg_sim('smooth one', 'polished item')

0.7569770216941833
1.3931821584701538


0.6965910792350769

In [ ]:
avg_sim('blue bag', 'blue ranch flavored bag')

0.9999999403953552
2.0000000596046448


1

In [ ]:
train_data = []
for _, row in train.iterrows():
  train_data.append([avg_sim(row.phrase1, row.phrase2), row.label])

test_data = []
for _, row in test.iterrows():
  test_data.append([avg_sim(row.phrase1, row.phrase2), row.label])



In [ ]:
import numpy as np

X_train = np.array(train_data)[:, 0].reshape(-1, 1)
y_train = np.array(train_data)[:, 1]

In [ ]:
import numpy as np

X_test = np.array(test_data)[:, 0].reshape(-1, 1)
y_test = np.array(test_data)[:, 1]

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

In [ ]:
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score

y_pred = model.predict(X_test)

In [ ]:
confusion_matrix(y_pred, y_test)

array([[510, 538],
       [490, 462]])

In [ ]:
precision_score(y_pred, y_test)

0.462

In [ ]:
recall_score(y_pred, y_test)

0.4852941176470588

In [ ]:
import nltk
nltk.download('wordnet')

from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
def content_words_similarity(phrase, sentence):
  phrase_nlp = nlp(phrase)
  sent_nlp = nlp(sentence)

  root_phrase = phrase.split()[-1]

  content_words = []
  for token in sent_nlp:
    if token.tag_ in ('NN', 'NNS', 'NNP', 'NNPS'):
      content_words.append(token.text)

  # print(content_words)
  ents = []
  for ent in sent_nlp.ents:
    ents.append(ent.root.text)

  content_words = [word for word in content_words if word not in ents and word != root_phrase]

  avg_similarity = None
  for synset1 in wn.synsets(root_phrase):
    avg_similarity = 0.0
    for word in content_words:
      word_similarity = 0.0
      for synset2 in wn.synsets(word):
        sim = synset1.wup_similarity(synset2)
        if sim > word_similarity:
          word_similarity = sim
      avg_similarity += word_similarity
      # print(avg_similarity)

  if avg_similarity is None:
    return None

  try:
    avg_similarity /= len(content_words)
  except ZeroDivisionError:
    return None
  # print(content_words)

  return avg_similarity

In [ ]:
content_words_similarity('one ensemble', 'Since 2004, the music department has taken one ensemble on a short UK tour in the summer term.')

0.26904761904761904

In [ ]:
content_words_similarity('a group', 'Since 2004, the music department has taken a group on a short UK tour in the summer term.')

0.18672438672438674

In [ ]:
train.shape

(7004, 6)

In [ ]:
train_data = []
for _, row in train.iterrows():
  train_data.append([avg_sim(row.phrase1, row.phrase2),
                     content_words_similarity(row.phrase1, row.sentence1),
                     content_words_similarity(row.phrase2, row.sentence2),
                     row.label])

test_data = []
for _, row in test.iterrows():
  test_data.append([avg_sim(row.phrase1, row.phrase2),
                     content_words_similarity(row.phrase1, row.sentence1),
                     content_words_similarity(row.phrase2, row.sentence2),
                    row.label])



Streaming output truncated to the last 5000 lines.
0.8328725099563599
1.6487417817115784
2.631838083267212
0.6341660618782043
1.4924601316452026
1.0
1.9291781783103943
0.9565293192863464
1.8917837142944336
0.6788355112075806
1.6057472229003906
0.7672176361083984
1.5640857815742493
0.5943479537963867
1.5367770791053772
0.960705578327179
1.6798242926597595
0.8676300644874573
1.6205393075942993
0.864837646484375
1.6699819564819336
0.9339301586151123
1.8896686434745789
0.9444263577461243
1.8422947525978088
0.931473970413208
1.7848221063613892
0.49604398012161255
1.3673151731491089
0.8164535164833069
1.7746678590774536
0.7807904481887817
1.580850601196289
0.6979660391807556
1.4020657539367676
0.8391744494438171
1.8391744494438171
0.8113111257553101
1.6174587607383728
0.937215268611908
1.8940686583518982
0.8226286172866821
1.578439474105835
0.9484949707984924
1.9090150594711304
0.818540096282959
1.4455148577690125
0.9284734725952148
1.7739282846450806
0.9395583271980286
1.8275826573371887
0.

In [ ]:
train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)

train_data = train_data.dropna()
test_data = test_data.dropna()

X_train = train_data.iloc[:, :-1].values
y_train = train_data.iloc[:, -1].values

X_test = test_data.iloc[:, :-1].values
y_test = test_data.iloc[:, -1].values

In [ ]:
import pickle

with open('x-train.pkl', 'wb') as f:
  pickle.dump(X_train, f)

with open('x-test.pkl', 'wb') as f:
  pickle.dump(X_test, f)

with open('y-train.pkl', 'wb') as f:
  pickle.dump(y_train, f)

with open('y-test.pkl', 'wb') as f:
  pickle.dump(y_test, f)

In [ ]:
model = LogisticRegression()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

confusion_matrix(y_pred, y_test)

array([[556, 555],
       [421, 423]])

In [ ]:
y_test

array([0, 0, 1, ..., 0, 0, 1])

In [ ]:
y_pred

array([1, 0, 0, ..., 1, 0, 0])

In [ ]:
X_train.shape

(98, 3)

In [ ]:
X_test.shape

(29, 3)

In [ ]:
precision_score(y_pred, y_test)

0.434560327198364

In [ ]:
recall_score(y_pred, y_test)

0.5

In [ ]:
model = SVC()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

confusion_matrix(y_pred, y_test)

array([[785, 752],
       [192, 226]])

In [ ]:
783 / (755+755)

0.5185430463576159

In [ ]:
783 / (783+194)

0.8014329580348004

In [ ]:
pd.DataFrame(zip(y_test, y_pred))

,0,1
0,0,0
1,0,0
2,1,0
3,0,0
4,0,0
...,...,...
1950,0,1
1951,0,0
1952,0,1
1953,0,0


In [ ]:
list(y_test).count(0)

977

In [ ]:
(783+223) / (783+755+194+223)

0.5145780051150896

In [ ]:
val_data = []
for _, row in val.iterrows():
  val_data.append([avg_sim(row.phrase1, row.phrase2),
                     content_words_similarity(row.phrase1, row.sentence1),
                     content_words_similarity(row.phrase2, row.sentence2),
                    row.label])

val_data = pd.DataFrame(val_data)

val_data = val_data.dropna()

X_val = val_data.iloc[:, :-1].values
y_val = val_data.iloc[:, -1].values

with open('x-val.pkl', 'wb') as f:
  pickle.dump(X_val, f)

with open('y-val.pkl', 'wb') as f:
  pickle.dump(y_val, f)

In [ ]:
X_val

array([[0.81372422, 0.17249417, 0.32492997],
       [1.45375019, 0.29444444, 0.30707283],
       [0.86132824, 0.18492618, 0.51470588],
       ...,
       [0.93644339, 0.40357143, 0.61912519],
       [0.94497019, 0.16233766, 0.35      ],
       [0.73985642, 0.18636364, 0.17189523]])

In [ ]:
val_data.describe()

,0,1,2,3
count,972.000000,972.000000,972.000000,972.000000
mean,0.865184,0.263864,0.285261,0.502058
std,0.097696,0.110261,0.115464,0.500253
min,0.379010,0.000000,0.000000,0.000000
25%,0.817130,0.192216,0.200162,0.000000
50%,0.872433,0.240823,0.267253,1.000000
75%,0.919899,0.311957,0.346151,1.000000
max,1.623142,1.000000,0.933333,1.000000


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)

y_pred = rfc.predict(X_test)

In [ ]:
confusion_matrix(y_pred, y_test)

array([[488, 577],
       [489, 401]])

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(3,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer with sigmoid activation for binary classification
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val))

Epoch 1/100
108/108 [==============================] - 2s 5ms/step - loss: 0.6936 - accuracy: 0.5031 - val_loss: 0.6939 - val_accuracy: 0.5000
Epoch 2/100
108/108 [==============================] - 0s 3ms/step - loss: 0.6933 - accuracy: 0.5050 - val_loss: 0.6939 - val_accuracy: 0.5021
Epoch 3/100
108/108 [==============================] - 0s 3ms/step - loss: 0.6938 - accuracy: 0.4999 - val_loss: 0.6934 - val_accuracy: 0.5093
Epoch 4/100
108/108 [==============================] - 0s 3ms/step - loss: 0.6932 - accuracy: 0.5004 - val_loss: 0.6941 - val_accuracy: 0.5000
Epoch 5/100
108/108 [==============================] - 0s 3ms/step - loss: 0.6934 - accuracy: 0.5122 - val_loss: 0.6945 - val_accuracy: 0.5041
Epoch 6/100
108/108 [==============================] - 0s 3ms/step - loss: 0.6938 - accuracy: 0.5039 - val_loss: 0.6939 - val_accuracy: 0.5041
Epoch 7/100
108/108 [==============================] - 0s 3ms/step - loss: 0.6934 - accuracy: 0.5026 - val_loss: 0.6934 - val_accuracy: 0.5093

In [ ]:
model.evaluate(X_test, y_test)

62/62 [==============================] - 0s 3ms/step - loss: 0.6931 - accuracy: 0.5018


[0.6930545568466187, 0.5017902851104736]

In [ ]:
y_pred = model.predict(X_test)

y_pred = [1 if i>0.5 else 0 for [i] in y_pred]

62/62 [==============================] - 0s 1ms/step


In [ ]:
confusion_matrix(y_pred, y_test)

array([[668, 665],
       [309, 313]])